This notebook uses a set of functions and module within the bispy package to interact with the GBIF API to pull back information on WLCI species.

The GBIF records are retrieved and cached based on valid/accepted ITIS species names and the invalid/not accepted ITIS species names.

In [1]:
#Import needed packages
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed

gbif = bispy.gbif.Gbif()
bis_utils = bispy.bis.Utils()

In [2]:
# Open up the cached updated ITIS list with valid/accepted species names from ITIS Exploration.ipynb
with open("cache/valid_itis.json", "r") as f:
    valid_itis = json.loads(f.read())

In [3]:
# Use joblib to run multiple requests for GBIF records in parallel via the valid/accepted ITIS scientific names
gbif_results = Parallel(n_jobs=8)(delayed(gbif.summarize_us_species)(name) for name in [r["scientific_name"] for r in valid_itis])
len(gbif_results)

116

In [4]:
# Filter to give just cases where GBIF species name matched with valid/accepted ITIS names
success_gbif = [i for i in gbif_results if i["processing_metadata"]["status"] == "success"]
len(success_gbif)

114

In [6]:
#Display records listed as failure for valid/accepted ITIS specie names
fail_gbif = [i for i in gbif_results if i["processing_metadata"]["status"] == "failure"]
fail_gbif

[{'processing_metadata': {'status': 'failure',
   'date_processed': '2019-08-14T14:35:26.619359',
   'status_message': 'Not Matched',
   'api': ['https://api.gbif.org/v1/species/suggest?q=Artemisia tridentata ssp. wyomingensis']},
  'parameters': {'Scientific Name': 'Artemisia tridentata ssp. wyomingensis'}},
 {'processing_metadata': {'status': 'failure',
   'date_processed': '2019-08-14T14:35:26.619359',
   'status_message': 'Not Matched',
   'api': ['https://api.gbif.org/v1/species/suggest?q=Oncorhynchus clarkii pleuriticus']},
  'parameters': {'Scientific Name': 'Oncorhynchus clarkii pleuriticus'}}]

In [7]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/gbif.json", success_gbif))

{'Doc Cache File': 'cache/gbif.json',
 'Number of Documents in Cache': 114,
 'Document Number 95': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-14T14:35:26.619359',
   'status_message': 'Matched',
   'api': ['https://api.gbif.org/v1/species/suggest?q=Fringilla montifringilla',
    'https://api.gbif.org/v1/occurrence/search?country=US&limit=0&facet=institutionCode&facet=year&facet=basisOfRecord&scientificName=Fringilla montifringilla']},
  'parameters': {'Scientific Name': 'Fringilla montifringilla'},
  'gbif_species': {'key': 2494441,
   'resolvable_identifier': 'http://api.gbif.org/v1/species/2494441',
   'biological_taxonomy': [{'rank': 'kingdom', 'name': 'Animalia'},
    {'rank': 'phylum', 'name': 'Chordata'},
    {'rank': 'class', 'name': 'Aves'},
    {'rank': 'order', 'name': 'Passeriformes'},
    {'rank': 'family', 'name': 'Fringillidae'},
    {'rank': 'genus', 'name': 'Fringilla'},
    {'rank': 'species', 'name': 'Fringilla montifringilla'}],
   'S

In [8]:
# Open up the cached updated ITIS list with invalid/not accepted species names from ITIS Exploration.ipynb
with open("cache/invalid_itis.json", "r") as f:
    invalid_itis = json.loads(f.read())

In [9]:
# Use joblib to run multiple requests for GBIF documents in parallel via the invalid/not accepted ITIS scientific names
invalid_result = Parallel(n_jobs=8)(delayed(gbif.summarize_us_species)(name) for name in [r["scientific_name"] for r in invalid_itis]) 

In [15]:
#Display result to identify the GBIF record that matched the invalid/not accepted ITIS name
invalid_result

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-14T14:35:26.619359',
   'status_message': 'Matched',
   'api': ['https://api.gbif.org/v1/species/suggest?q=Amphispiza belli',
    'https://api.gbif.org/v1/occurrence/search?country=US&limit=0&facet=institutionCode&facet=year&facet=basisOfRecord&scientificName=Amphispiza belli']},
  'parameters': {'Scientific Name': 'Amphispiza belli'},
  'gbif_species': {'key': 2491751,
   'resolvable_identifier': 'http://api.gbif.org/v1/species/2491751',
   'biological_taxonomy': [{'rank': 'kingdom', 'name': 'Animalia'},
    {'rank': 'phylum', 'name': 'Chordata'},
    {'rank': 'class', 'name': 'Aves'},
    {'rank': 'order', 'name': 'Passeriformes'},
    {'rank': 'family', 'name': 'Emberizidae'},
    {'rank': 'genus', 'name': 'Artemisiospiza'},
    {'rank': 'species', 'name': 'Artemisiospiza belli'}],
   'Scientific Name': 'Amphispiza belli',
   'name_with_source': 'Amphispiza belli (Cassin, 1850)',
   'rank': 'SPECIES',
   'T

In [ ]:
# Cache the retrieved document and return/display a sample for verification
display(bis_utils.doc_cache("cache/Results of Consultations with Invalid ITIS Species Names/invalid_gbif.json", invalid_result))